In [ ]:
# Instalando as dependências do SINAPSES
#!pip install -r /content/requirements.txt

In [87]:
import logging
import sys
from typing import Callable
from sinapses_cliente.modelo.binario import Binario
from sinapses_cliente.modelo.modelo_versao_recurso import ModeloVersaoRecurso
from sinapses_cliente.modelo.resposta.resposta_classificacao import Resposta
from sinapses_cliente.modelo.servico_contexto import ServicoContexto
from sinapses_cliente.modelo.tipo_conteudo import TipoConteudo
from sinapses_cliente.modelo.treinamento_resultado import TreinamentoResultado
from sinapses_cliente.sinapses_pipeline import SinapsesPipeline
from sinapses_cliente.sinapses_sessao import SinapsesSessao
from sinapses_cliente.modelo.requisicao.mensagem import Mensagem

import json
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt
import joblib

In [88]:
def funcao_treinamento(sessao: SinapsesSessao) -> TreinamentoResultado:
    # === 1. Carregar dados ===
    with open("dados_salvos.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    # === 2. Criar DataFrame com textos dos movimentos ===
    records = []
    for item in data["hits"]["hits"]:
        proc = item["_source"]
        numero = proc.get("numeroProcesso")
        classe = proc.get("classe", {}).get("nome", "")
        assunto = "; ".join(a["nome"] for a in proc.get("assuntos", []) if "nome" in a)
        movimentos = " > ".join(m["nome"] for m in proc.get("movimentos", []) if "nome" in m)

        records.append({
            "numeroProcesso": numero,
            "classe": classe,
            "assunto": assunto,
            "movimentos_texto": movimentos
        })

    df = pd.DataFrame(records)

    # === 3. Vetorizar os movimentos ===
    vectorizer = TfidfVectorizer()
    X_text = vectorizer.fit_transform(df["movimentos_texto"])

    # === 4. Gerar rótulos com Isolation Forest ===
    modelo_iso = IsolationForest(contamination=0.1, random_state=42)
    y_iso = modelo_iso.fit_predict(X_text)

    # Normalizar rótulo para 0 (normal) e 1 (anomalia)
    df["anomalia"] = np.where(y_iso == -1, 1, 0)

    # === 5. Preparar dados para rede neural ===
    X = X_text.toarray().astype(np.float32)
    y = df["anomalia"].values.astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # === 6. Definir rede neural ===
    modelo = Sequential([
        Dense(128, activation='relu', input_shape=(X.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # saída binária
    ])

    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # === 7. Treinar ===
    early_stop = EarlyStopping(patience=5, restore_best_weights=True)
    history = modelo.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stop])

    # === 8. Avaliação ===
    loss, accuracy = modelo.evaluate(X_test, y_test, verbose=0)

    y_pred = (modelo.predict(X_test) > 0.5).astype("int32")
    relatorio = classification_report(y_test, y_pred, target_names=["normal", "anomalia"])

    with open("relatorio_rede_neural_tfidf.txt", "w", encoding="utf-8") as f:
        f.write(relatorio)

    # === 9. Salvar modelo e vetor TF-IDF ===
    modelo.save("modelo_rede_neural_tfidf.h5")

    joblib.dump(vectorizer, "vetorizador_tfidf.pkl")

    # === 10. Gráficos ===
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('Perda')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.title('Acurácia')
    plt.legend()

    plt.tight_layout()
    plt.savefig("treinamento_rede_neural_tfidf.png")
    plt.close()

    # === Salvar arquitetura do modelo em JSON ===
    modelo_json = modelo.to_json()
    with open("modelo.json", "w", encoding="utf-8") as json_file:
        json_file.write(modelo_json)

    # print("Rede neural treinada com vetores TF-IDF e rótulos do Isolation Forest.")

    # Retornar os resultados
    return TreinamentoResultado(accuracy, [
        ModeloVersaoRecurso('treinamento_rede_neural_tfidf.png',
                            Binario.from_arquivo('treinamento_rede_neural_tfidf.png'),
                            TipoConteudo.IMAGE_PNG,
                            'GRAFICO_TREINAMENTO'),
        ModeloVersaoRecurso('dados_salvos.json', Binario.from_arquivo('dados_salvos.json'),
                            TipoConteudo.APPLICATION_JSON,
                            'DADOS_TREINAMENTO'),
        ModeloVersaoRecurso('modelo.json', Binario(bytes(modelo_json, 'UTF-8')), TipoConteudo.APPLICATION_JSON,
                            'MODELO_TREINADO'),
        ModeloVersaoRecurso('modelo.pesos.h5', Binario.from_arquivo('modelo_rede_neural_tfidf.h5'),
                            TipoConteudo.APPLICATION_OCTET_STREAM, 'MODELO_TREINADO')
    ])

In [89]:
def funcao_inicializacao_servico(sessao: SinapsesSessao):
    modelo_versao = sessao.get_modelo_versao()
    modelo = model_from_json(modelo_versao.get_recurso_por_nome('modelo.json').conteudo.bytes)
    arquivo_modelo_weights = str(sessao.get_diretorio_trabalho()) + '/modelo_weights.h5'
    modelo_versao.get_recurso_por_nome('modelo.pesos.h5').conteudo.salvar_em(arquivo_modelo_weights)
    modelo.load_weights(arquivo_modelo_weights)

    logger = logging.getLogger('Sinapses')
    logger.setLevel(logging.ERROR)
    formatter = logging.Formatter(fmt='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    stream_handler = logging.StreamHandler(sys.stdout)
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    return ServicoContexto(
        modelo=modelo,
        logger=logger
    )

In [90]:
def funcao_servico(sessao: SinapsesSessao, ctx: ServicoContexto, requisicao: dict) -> Resposta:
    resultado = ""

    # Carregar o vetorizador apenas uma vez
    vectorizer = joblib.load("vetorizador_tfidf.pkl")

    # Extrair os movimentos do dicionário da requisição
    movimentos_raw = requisicao.get("movimentos", [])

    # Reorganizar para obter os nomes dos movimentos
    movimentos = []
    for mov in movimentos_raw:
        nome_info = mov.get("nome", {})
        if nome_info.get("tipo") == "TEXTO_PURO" and "conteudo" in nome_info:
            movimentos.append(nome_info["conteudo"])

    if len(movimentos) < 4:
        resultado = "Não foi possível avaliar"
    else:
        # Concatenar nomes com separador
        texto_movimentos = " > ".join(movimentos)

        # Vetorizar o texto com TF-IDF
        X_novo = vectorizer.transform([texto_movimentos]).toarray()

        # Fazer a predição com o modelo
        pred = ctx.modelo.predict(X_novo)[0][0]

        # Interpretar o resultado
        if pred >= 0.5:
            resultado = "Possui anomalias na movimentação"
        else:
            resultado = "Não possui anomalias na movimentação"

    # print(resultado)

    return Resposta(**{"resultado": resultado})

In [91]:
def funcao_teste_servico(_s: SinapsesSessao, _c: ServicoContexto, servico: Callable[[dict], dict]) -> None:
    r1 = servico({
        "classe": Mensagem.texto("Procedimento do Juizado Especial Cível").to_dict(),
        "assunto": Mensagem.texto("Pensão por Morte (Art. 74/9)").to_dict(),
        "numeroProcesso": Mensagem.texto("50013586020254025103").to_dict(),
        "movimentos": [
            {
                "codigo": Mensagem.texto("26").to_dict(),
                "nome": Mensagem.texto("Distribuição").to_dict(),
                "dataHora": Mensagem.texto("2025-02-26T13:02:29.000Z").to_dict()
            },
            {
                "codigo": Mensagem.texto("12164").to_dict(),
                "nome": Mensagem.texto("Outras Decisões").to_dict(),
                "dataHora": Mensagem.texto("2025-03-10T18:09:31.000Z").to_dict()
            },
            {
                "codigo": Mensagem.texto("11010").to_dict(),
                "nome": Mensagem.texto("Mero expediente").to_dict(),
                "dataHora": Mensagem.texto("2025-03-21T17:25:24.000Z").to_dict()
            },
            {
                "codigo": Mensagem.texto("11010").to_dict(),
                "nome": Mensagem.texto("Mero expediente").to_dict(),
                "dataHora": Mensagem.texto("2025-03-25T21:20:27.000Z").to_dict()
            },
            {
                "codigo": Mensagem.texto("898").to_dict(),
                "nome": Mensagem.texto("Por decisão judicial").to_dict(),
                "dataHora": Mensagem.texto("2025-03-26T23:19:27.000Z").to_dict()
            },
            {
                "codigo": Mensagem.texto("12067").to_dict(),
                "nome": Mensagem.texto("Levantamento da Suspensão ou Dessobrestamento").to_dict(),
                "dataHora": Mensagem.texto("2025-04-30T18:33:56.000Z").to_dict()
            }
        ]
    })

    r2 = servico(
        {
            "classe": Mensagem.texto("Cumprimento de Sentença contra a Fazenda Pública").to_dict(),
            "assunto": Mensagem.texto("Rural (Art. 48/51)").to_dict(),
            "numeroProcesso": Mensagem.texto("50001245220254025003").to_dict(),
            "movimentos": [
                {
                    "codigo": Mensagem.texto("26").to_dict(),
                    "nome": Mensagem.texto("Distribuição").to_dict(),
                    "dataHora": Mensagem.texto("2025-01-15T23:18:51.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("85").to_dict(),
                    "nome": Mensagem.texto("Petição").to_dict(),
                    "dataHora": Mensagem.texto("2025-03-28T08:13:40.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("85").to_dict(),
                    "nome": Mensagem.texto("Petição").to_dict(),
                    "dataHora": Mensagem.texto("2025-03-30T17:02:46.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("85").to_dict(),
                    "nome": Mensagem.texto("Petição").to_dict(),
                    "dataHora": Mensagem.texto("2025-03-31T14:44:00.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("10966").to_dict(),
                    "nome": Mensagem.texto("Mudança de Classe Processual").to_dict(),
                    "dataHora": Mensagem.texto("2025-03-31T10:26:13.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("11010").to_dict(),
                    "nome": Mensagem.texto("Mero expediente").to_dict(),
                    "dataHora": Mensagem.texto("2025-01-23T16:08:02.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("466").to_dict(),
                    "nome": Mensagem.texto("Homologação de Transação").to_dict(),
                    "dataHora": Mensagem.texto("2025-01-29T16:15:25.000Z").to_dict()
                }
            ]
        }
    )

    r3 = servico(
        {
            "classe": Mensagem.texto("Procedimento do Juizado Especial Cível").to_dict(),
            "assunto": Mensagem.texto("Pensão por Morte (Art. 74/9)").to_dict(),
            "numeroProcesso": Mensagem.texto("50013906520254025103").to_dict(),
            "movimentos": [
                {
                    "codigo": Mensagem.texto("26").to_dict(),
                    "nome": Mensagem.texto("Distribuição").to_dict(),
                    "dataHora": Mensagem.texto("2025-02-27T10:38:49.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("12164").to_dict(),
                    "nome": Mensagem.texto("Outras Decisões").to_dict(),
                    "dataHora": Mensagem.texto("2025-03-12T17:37:48.000Z").to_dict()
                },
                {
                    "codigo": Mensagem.texto("11010").to_dict(),
                    "nome": Mensagem.texto("Mero expediente").to_dict(),
                    "dataHora": Mensagem.texto("2025-04-07T13:19:04.000Z").to_dict()
                }
            ]
        }
    )

    assert r1 is not None
    assert r2 is not None
    assert r3 is not None

In [101]:
#import __main__ as main
#main.__file__ = "ia_revisor_inteligente.ipynb"

In [102]:
# Inicializar o pipeline
pipeline = SinapsesPipeline.instancia()

O arquivo de configuração foi encontrado em: /content/sinapses.yml
Validando as configurações do arquivo encontrado...
 - O endereço do sinapses é válido!
 - A autenticação é válida!
 - O id da versão do modelo é válida!
Arquivo de configuração válido!
ERRO: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


In [103]:
# Definir as funções
pipeline.definir_treinamento(funcao_treinamento)
pipeline.definir_inicializacao_servico(funcao_inicializacao_servico)
pipeline.definir_servico(funcao_servico)
pipeline.definir_teste_servico(funcao_teste_servico)

In [ ]:
# testar pipeline
# pipeline.testar_pipeline()

In [104]:
# Envia o código do modelo para o SINAPSES
pipeline.enviar_treinado()

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8455 - loss: 0.6176 - val_accuracy: 0.9125 - val_loss: 0.4040
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8991 - loss: 0.3513 - val_accuracy: 0.9125 - val_loss: 0.2167
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9043 - loss: 0.2182 - val_accuracy: 0.9125 - val_loss: 0.1707
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9076 - loss: 0.1897 - val_accuracy: 0.9438 - val_loss: 0.1382
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9503 - loss: 0.1183 - val_accuracy: 0.9563 - val_loss: 0.1194
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9737 - loss: 0.0950 - val_accuracy: 0.9563 - val_loss: 0.1067
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9837 - loss: 0.0676 - val_accuracy: 0.9563 - val_loss: 0.1026
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9839 - loss: 0.0681 - val_accuracy: 0.9625 - val_loss: 0.10

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
